In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import os

notebook_path = os.path.abspath('')
import sys
# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'ensemble_commit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        print(commit_fit_path)
        break

if commit_fit_path is None:
    raise ValueError("Path containing 'ensemble_commit' not found in notebook path.")


if commit_fit_path not in sys.path:
    sys.path.append(commit_fit_path)

import ensemble_model.preprocesser as preprocesser 
import ensemble_model.MoE_model_loss as moe 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer, DistilBertModel

# import whatthepatch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve,classification_report



root
ensemble_commit
/root/ensemble_commit


In [2]:
moe.BiLSTMStudent

ensemble_model.MoE_model_loss.BiLSTMStudent

In [3]:
df = pd.read_csv(r'../datasets/dataset.csv', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'negative':0,'positive':1}
df = df.replace({"label": label2id})
df

/tmp/ipykernel_3748/1052552286.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,Unnamed: 0,label,message,diff
0,0,0,Merge pull request #46 from rufferson/saslx-tl...,diff --git a/lib/DJabberd.pm b/lib/DJabberd.pm...
1,1,1,Fix leaks in kadmin server stubs [CVE-2015-863...,diff --git a/src/kadmin/server/server_stubs.c ...
2,2,1,Validate authorization request on approval\n\n...,diff --git a/spring-security-oauth2/src/main/j...
3,3,1,Release 2.72.4+171110,diff --git a/application/config/version.php b/...
4,4,0,Fixing compiler warnings.,diff --git a/src/main.c b/src/main.c\nindex 50...
...,...,...,...,...
10107,10107,0,les: remove useless protocol defines (#22115)\...,diff --git a/les/benchmark.go b/les/benchmark....
10108,10108,1,Merge pull request #2067 from realm/tg-swift-l...,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...
10109,10109,0,[fix] 新規アカウント作成ページの翻訳を修正,diff --git a/app/locales/ja_JP/LC_MESSAGES/mes...
10110,10110,0,Merge branch 'hotfixes',diff --git a/CHANGELOG.txt b/CHANGELOG.txt\nin...


In [4]:
# df['command'] = df['diff'].apply(lambda x : extract_lines(find_longest_list([y.changes for y in whatthepatch.parse_patch(x)])))
# extracted_cols = df['diff'].str.extract(r'^diff --git (?P<source>[^\t\n]+) (?P<target>[^\t\n]+)')
# whatthepatch.apply_diff(diff, lao)

In [5]:
# len(df_dataset)

In [6]:
# df_dataset[0]

In [7]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('../models/bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('../models/bert-base-cased')

codebert_model = BertModel.from_pretrained('../models/bert-base-cased')
codebert_tokenizer = BertTokenizer.from_pretrained('../models/bert-base-cased')

Some weights of the model checkpoint at ../models/bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ../models/bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 

In [8]:
train_data, _ = train_test_split(df, train_size=0.2, random_state=42)
train_data, test_data = train_test_split(train_data, test_size=0.3, random_state=42)
# train_data = pd.read_csv('./datasets/PD_train.csv')
# test_data = pd.read_csv('./datasets/PD_test.csv')
# val_data = pd.read_csv('./datasets/PD_val.csv')



In [9]:
len(train_data)

1408

In [10]:
# codebert_model.config.hidden_size

In [11]:
len(test_data)

604

In [12]:

test_data, val_data = train_test_split(test_data, train_size=0.5, random_state=42)

In [13]:
train_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)
val_data.reset_index(inplace=True)

In [14]:
train_data['label'].value_counts()

label
0    860
1    548
Name: count, dtype: int64

In [15]:
val_data

,index,Unnamed: 0,label,message,diff
0,1066,1066,1,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/psd.c b/coders/psd.c\ninde...
1,5783,5783,0,No way this will be a 1.2 release.,diff --git a/php_radius.h b/php_radius.h\ninde...
2,5137,5137,0,Cleanups,diff --git a/src/TopicTree.h b/src/TopicTree.h...
3,2488,2488,1,Ensure token expiration is maintained\n\nEnsur...,diff --git a/keystone/service.py b/keystone/se...
4,6686,6686,1,Add end string anchor to string origin def\n\n...,diff --git a/lib/rack/cors.rb b/lib/rack/cors....
...,...,...,...,...,...
297,857,857,0,Add plugin for rethinkdb. Rethinkdb can find o...,diff --git a/ajenti/plugins/rethink/__init__.p...
298,6144,6144,0,release date in manual,diff --git a/docs/index.html b/docs/index.html...
299,3701,3701,0,Make Array.prototype.unshift method conform to...,diff --git a/jerry-core/ecma/builtin-objects/e...
300,4271,4271,1,Explicitly configure allowed credential types\...,diff --git a/java/org/apache/catalina/mbeans/J...


In [16]:
val_data

,index,Unnamed: 0,label,message,diff
0,1066,1066,1,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/psd.c b/coders/psd.c\ninde...
1,5783,5783,0,No way this will be a 1.2 release.,diff --git a/php_radius.h b/php_radius.h\ninde...
2,5137,5137,0,Cleanups,diff --git a/src/TopicTree.h b/src/TopicTree.h...
3,2488,2488,1,Ensure token expiration is maintained\n\nEnsur...,diff --git a/keystone/service.py b/keystone/se...
4,6686,6686,1,Add end string anchor to string origin def\n\n...,diff --git a/lib/rack/cors.rb b/lib/rack/cors....
...,...,...,...,...,...
297,857,857,0,Add plugin for rethinkdb. Rethinkdb can find o...,diff --git a/ajenti/plugins/rethink/__init__.p...
298,6144,6144,0,release date in manual,diff --git a/docs/index.html b/docs/index.html...
299,3701,3701,0,Make Array.prototype.unshift method conform to...,diff --git a/jerry-core/ecma/builtin-objects/e...
300,4271,4271,1,Explicitly configure allowed credential types\...,diff --git a/java/org/apache/catalina/mbeans/J...


In [17]:


# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train_data, bert_tokenizer, codebert_tokenizer, message='message',command='diff',label='label')
val_dataset = preprocesser.SentencePairDataset(val_data, bert_tokenizer, codebert_tokenizer, message='message',command='diff',label='label')
test_dataset = preprocesser.SentencePairDataset(test_data, bert_tokenizer, codebert_tokenizer, message='message',command='diff',label='label')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)



# # Initialize the model
# model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# # Train the model
# model.trainer(train_loader, val_loader,num_epochs=10)
# for batch in train_loader:
#     print(batch)

In [18]:
base_model1 = moe.BaseModel(bert_model)
base_model2 = moe.BaseModel(codebert_model)

In [19]:
# Load BERT and CodeBERT models and tokenizers
bert_model = DistilBertModel.from_pretrained('/root/autodl-tmp/IPCK/model/distilbert')

codebert_model = DistilBertModel.from_pretrained('/root/autodl-tmp/IPCK/model/distilbert')

Some weights of the model checkpoint at /root/autodl-tmp/IPCK/model/distilbert were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /root/autodl-tmp/IPCK/model/distilbert were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you

In [20]:
encoder_model1 = moe.EncoderModel(bert_model)
encoder_model2 = moe.EncoderModel(bert_model)

In [21]:


# Create stacking model
moe_model = moe.MoEModel(base_model1, base_model2)
# Train the model
moe_model.trainer(train_loader, val_loader,num_epochs=30, patience=3)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/30 Loss: 0.1571: 100%|██████████| 176/176 [03:35<00:00,  1.23s/batch]


=============================train========================
Validation Accuracy: 0.7715


Epoch 2/30 Loss: 0.1317: 100%|██████████| 176/176 [03:33<00:00,  1.22s/batch]


=============================train========================
Validation Accuracy: 0.7947


Epoch 3/30 Loss: 0.1060: 100%|██████████| 176/176 [03:30<00:00,  1.20s/batch]


...

=============================train========================
Validation Accuracy: 0.8477


Epoch 11/30 Loss: 0.0313: 100%|██████████| 176/176 [03:32<00:00,  1.20s/batch]


=============================train========================
Validation Accuracy: 0.8477


Epoch 12/30 Loss: 0.0337: 100%|██████████| 176/176 [03:35<00:00,  1.22s/batch]

=============================train========================
Early stopping triggered after 12 epochs


In [22]:
test_dataset = preprocesser.SentencePairDataset(test_data, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [23]:
torch.save(moe_model, "entire_bert_model_900repo.pth")

In [24]:
moe_model = torch.load("entire_bert_model_900repo.pth")                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [25]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = moe_model.evaluate(test_loader)

Validation Accuracy: 0.8411


In [26]:
print(classification_report(test_labels,test_predictions))

              precision    recall  f1-score   support

           0       0.88      0.86      0.87       189
           1       0.78      0.81      0.79       113

    accuracy                           0.84       302
   macro avg       0.83      0.83      0.83       302
weighted avg       0.84      0.84      0.84       302



In [27]:
test_acc

0.8410596026490066

In [28]:
student_model = moe.BiLSTMStudent(hidden_dim=256,output_dim =2, base_model1=encoder_model1,base_model2=encoder_model2)

In [29]:
# train_student_model

In [30]:
student_model.distill_trainer(moe_model, test_loader, num_epochs=10)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/10 Loss: 1.9687: 100%|██████████| 38/38 [00:36<00:00,  1.05batch/s]


=============================train========================
Validation Accuracy: 0.7252


Epoch 2/10 Loss: 1.3159: 100%|██████████| 38/38 [00:36<00:00,  1.04batch/s]


=============================train========================
Validation Accuracy: 0.8212


Epoch 3/10 Loss: 0.6481: 100%|██████████| 38/38 [00:36<00:00,  1.04batch/s]


...

=============================train========================
Validation Accuracy: 0.8311


Epoch 9/10 Loss: 0.0501: 100%|██████████| 38/38 [00:36<00:00,  1.04batch/s]


=============================train========================
Validation Accuracy: 0.8510


Epoch 10/10 Loss: 0.0372: 100%|██████████| 38/38 [00:36<00:00,  1.04batch/s]


=============================train========================
Validation Accuracy: 0.8411


In [31]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = student_model.evaluate(val_loader)

Validation Accuracy: 0.7351


In [32]:
print(classification_report(test_labels,test_predictions))

              precision    recall  f1-score   support

           0       0.82      0.71      0.76       182
           1       0.64      0.77      0.70       120

    accuracy                           0.74       302
   macro avg       0.73      0.74      0.73       302
weighted avg       0.75      0.74      0.74       302



In [33]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = moe_model.evaluate(val_loader)

Validation Accuracy: 0.8377


In [34]:
print(classification_report(test_labels,test_predictions))

              precision    recall  f1-score   support

           0       0.85      0.88      0.87       182
           1       0.81      0.77      0.79       120

    accuracy                           0.84       302
   macro avg       0.83      0.83      0.83       302
weighted avg       0.84      0.84      0.84       302



In [35]:
torch.save(student_model, "student_model_900repo.pth")

In [36]:
#发送多种类型的邮件
from email.mime.multipart import MIMEMultipart
import smtplib

from email.mime.text import MIMEText
msg_from = '915803745@qq.com'  # 发送方邮箱
passwd = 'vcuosuurrgkfbdai'   #就是上面的授权码
 
# to= ['g.zhang@gotion.com', 'j.tong@gotion.com'] #接受方邮箱
to= ['j.tong@gotion.com'] #接受方邮箱
#设置邮件内容
#MIMEMultipart类可以放任何内容
msg = MIMEMultipart()
conntent="在线模型训练完毕"
#把内容加进去
msg.attach(MIMEText(conntent,'plain','utf-8'))
 
#设置邮件主题
msg['Subject']="在线模型训练完毕"
 
#发送方信息
msg['From']=msg_from
 
#开始发送
 
#通过SSL方式发送，服务器地址和端口
s = smtplib.SMTP_SSL("smtp.qq.com", 465)
# 登录邮箱
s.login(msg_from, passwd)
#开始发送
s.sendmail(msg_from,to,msg.as_string())
print("在线模型训练完毕")

在线模型训练完毕
